## 使用腾讯开源Word2vec实现内容相似推荐

下载地址：https://ai.tencent.com/ailab/nlp/embedding.html

数据格式：
* 下载后解压文件名为Tencent_AILab_ChineseEmbedding.txt
* 第一行是8824330 200，即行数和维度数
* 之后的每行，用空格分隔201列，其中第一个是词语，后面是Embedding


实现步骤：
1. 获取文章列表数据，包括ID、标题、内容
2. 使用jieba实现关键词提取
3. 查询腾讯word2vec，平均法得到文档embedding
4. 对于输入的ID，计算最相似的文章列表

### 1. 获取数据

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("./datas/crazyant_blog_articles_wordsegs.csv")
df.head()

,id,post_title,words
0,78,JavaScript对Select的子元素Option的操作,javascript select 元素 option 操作 javascript 删除 所...
1,83,当用header方法输出内容时出现“Cannot modify header informa...,当用 header 方法 输出 内容 出现 cannot modify header inf...
2,85,linux下禁止机箱蜂鸣方法,linux 禁止 机箱 蜂鸣 方法 图像 界面 注意 图形界面 即使 图像 界面 快捷键 出...
3,87,硬盘分区表丢失、修复大事记--分区表修复利器testdisk,硬盘分区 丢失 修复 大事记 -- 分区表 修复 利器 testdisk 今天 2009 1...
4,91,vi编辑器命令,vi 编辑器 命令 vi 编辑器 文字说明 模式 命令 模式 编辑 模式 末行 模式 切换 ...


### 2. 查询腾讯数据集计算文档Embedding

#### 去重后的词表

In [3]:
all_words = set()
for idx, row in df.iterrows():
    all_words.update(row["words"].split())
len(all_words)

14871

#### 查询候选词语的腾讯wordEmbedding

In [5]:
# 候选词语的embedding
word_embedding = {}

is_first_line = True
# with open("/home/pss/windir/Tencent_AILab_ChineseEmbedding.txt") as fin:
with open("./datas/small_tencent_embedding.txt") as fin:
    for line in fin:
        if is_first_line: 
            is_first_line = False
            continue
        fields = line[:-1].split()
        if len(fields) != 201:
            continue
        word = fields[0]
        if word in all_words:
            word_embedding[word] = np.array([float(x) for x in fields[1:]])

In [6]:
print(len(word_embedding), word_embedding["python"])

11382 [ 0.382475  0.156139  0.062302  0.030984  0.481495 -0.070564  0.466484
  0.546035 -0.238761 -0.126754  0.063588  0.091173 -0.272091 -0.113122
  0.170998 -0.139973 -0.335421 -0.1836   -0.119432 -0.026666 -0.524403
 -0.290163 -0.495772  0.136166  0.074637  0.126304 -0.41948   0.271968
  0.447399  0.041005 -0.180788  0.101531  0.333039  0.013628  0.052285
 -0.433536  0.494823 -0.29307  -0.073794  0.260313 -0.052784 -0.226162
  0.791509 -0.203396  0.059075 -0.082627 -0.423575 -0.371575 -0.568254
  0.134853  0.174628  0.138118  0.059341 -0.405463  0.452836 -0.067161
  0.236258  0.347981  0.16007  -0.213742  0.047517  0.232295 -0.151182
  0.011967 -0.528855 -0.566595  0.287433  1.006496  0.00421   0.031725
 -0.543762  0.194417  0.028565 -0.086813  0.005247  0.582514 -0.241997
  0.249292  0.14645   0.213583  0.241735 -0.29941  -0.192367  0.115289
  0.193353 -0.717468 -0.139603  0.110287 -0.104537  0.076995  0.669608
  0.192023 -0.157454  0.015251  0.42727   0.442922 -0.302943 -0.308798


#### 计算每篇文章的doc embedding

In [7]:
def compute_doc_vector(words):
    words = words.split()
    embeddings = []
    for word in words:
        if word in word_embedding:
            embeddings.append(word_embedding[word])
    return np.sum(embeddings, axis=0) / len(embeddings)

df["doc_vector"] = df["words"].map(compute_doc_vector)

In [8]:
df.head(3)

,id,post_title,words,doc_vector
0,78,JavaScript对Select的子元素Option的操作,javascript select 元素 option 操作 javascript 删除 所...,"[-0.013126199999999996, -0.03166106249999995, ..."
1,83,当用header方法输出内容时出现“Cannot modify header informa...,当用 header 方法 输出 内容 出现 cannot modify header inf...,"[0.173896275, -0.12689985, -0.0503381250000000..."
2,85,linux下禁止机箱蜂鸣方法,linux 禁止 机箱 蜂鸣 方法 图像 界面 注意 图形界面 即使 图像 界面 快捷键 出...,"[0.17765841463414633, -0.13135253658536586, -0..."


### 4. 对于给定文章算出最相似的10篇文章

In [9]:
# 随便挑选一篇文章ID，2583：pandas，581：PHP
article_id = 2583
df.loc[df["id"]==article_id]

,id,post_title,words,doc_vector
256,2583,Pandas实现数据的合并concat,pandas 实现 数据 合并 concat 使用 场景 批量 合并 相同 格式 excel...,"[0.005996356060606063, 0.0679557373737372, -0...."


In [10]:
article_embedding = df.loc[df["id"]==article_id, "doc_vector"].iloc[0]
article_embedding

array([ 5.99635606e-03,  6.79557374e-02, -5.15264015e-03, -7.76245278e-02,
        2.70167614e-02,  4.68369609e-02,  2.74888652e-01,  2.62035790e-01,
        5.88677449e-02, -6.58535985e-03,  1.11688506e-01,  1.56684524e-01,
        1.77044933e-01, -7.72565922e-02,  2.23586165e-01, -2.68098207e-02,
       -5.74492298e-02, -2.22931782e-01, -1.58685366e-01,  1.63666288e-02,
       -1.97521905e-01,  3.95486010e-02, -2.23003990e-02,  4.53217731e-01,
        4.97356540e-02,  1.55571486e-01,  1.22501005e-01,  1.97476905e-01,
        1.43449737e-01,  3.12355328e-01,  5.28948119e-02,  2.37558862e-01,
       -2.46354255e-02,  2.88600253e-02, -1.40968886e-01, -3.14568242e-01,
        4.63498718e-01, -6.93908093e-02, -2.77842109e-01, -1.02786698e-01,
        2.17917083e-02, -3.95267639e-02,  8.89407037e-01,  2.16067168e-01,
        2.37763532e-01,  1.08071389e-02, -6.36008851e-02, -3.85960957e-01,
       -9.01793687e-02, -5.02437525e-02, -1.93142354e-01,  1.20377753e-01,
       -2.66069100e-01,  

In [11]:
# 余弦相似度
from scipy.spatial import distance
df["sim_value"] = df["doc_vector"].map(lambda x : 1 - distance.cosine(article_embedding, x))

In [12]:
df[["id", "post_title", "sim_value"]].head(3)

,id,post_title,sim_value
0,78,JavaScript对Select的子元素Option的操作,0.953236
1,83,当用header方法输出内容时出现“Cannot modify header informa...,0.868788
2,85,linux下禁止机箱蜂鸣方法,0.783414


In [13]:
# 按相似度降序排列，查询前10条
df.sort_values(by="sim_value", ascending=False)[["id", "post_title", "sim_value"]].head(10)

,id,post_title,sim_value
256,2583,Pandas实现数据的合并concat,1.000000
252,2546,Pandas的axis参数怎么理解？,0.988473
255,2574,Pandas的Index索引有什么用途？,0.987658
259,2594,Pandas怎样实现对数据的分组统计？,0.987287
240,2502,Pandas系列-DataFrame和Series数据结构,0.979710
239,2499,Pandas系列-读取csv/excel/mysql数据,0.978739
262,2604,Pandas怎样对每个分组应用apply函数?,0.978439
258,2591,怎样使用Pandas批量拆分与合并Excel文件？,0.972822
10,107,PHP输出Excel实例代码,0.970517
30,151,PHP操作EXCEL相关,0.968934
